In [6]:
from flask import Flask
from flask_sqlalchemy import SQLAlchemy
import datetime
from datetime import datetime
import json

app = Flask(__name__)
database = 'sqlite:///database1.db'
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
app.config['SQLALCHEMY_DATABASE_URI'] = database
app.config['SECRET_KEY'] = 'SECRET_KEY'
db = SQLAlchemy(app)

class Todo(db.Model):
    __tablename__ = 'todos'

    id = db.Column(db.Integer, primary_key=True)
    _title = db.Column(db.String(255), unique=False, nullable=False)
    _description = db.Column(db.String(255), unique=False, nullable=True)
    _completed = db.Column(db.Boolean, default=False)

    def __init__(self, title, description=None, completed=False):
        self._title = title
        self._description = description
        self._completed = completed

    @property
    def title(self):
        return self._title

    @title.setter
    def title(self, title):
        self._title = title

    @property
    def description(self):
        return self._description

    @description.setter
    def description(self, description):
        self._description = description

    @property
    def completed(self):
        return self._completed

    @completed.setter
    def completed(self, completed):
        self._completed = completed

    def __str__(self):
        return json.dumps(self.read())

    def create(self):
        db.session.add(self)
        db.session.commit()
        return self

    def read(self):
        return {
            "id": self.id,
            "title": self.title,
            "description": self.description,
            "completed": self.completed,
        }

    def update(self, title="", description="", completed=None):
        if len(title) > 0:
            self.title = title
        if len(description) > 0:
            self.description = description
        if completed is not None:
            self.completed = completed
        db.session.commit()
        return self

    def delete(self):
        db.session.delete(self)
        db.session.commit()
        return None

def initTodos():
    with app.app_context():
        db.create_all()


initTodos()


def find_by_id(todo_id):
    with app.app_context():
        todo = Todo.query.filter_by(id=todo_id).first()
    return todo


def delete_by_id(todo_id):
    with app.app_context():
        todo = Todo.query.filter_by(id=todo_id).first()
        if todo:
            todo.delete()
            print(f"To-Do item with ID {todo_id} has been deleted.")
        else:
            print(f"To-Do item with ID {todo_id} not found.")


def create_todo():
    title = input("Enter the title of the To-Do item: ")
    description = input("Enter a description for the To-Do item (optional): ")
    if description == "":
        description = None

    todo = Todo(title=title, description=description)

    with app.app_context():
        try:
            object = todo.create()
            print("Created\n", object.read())
        except:
            print(f"An error occurred while creating the To-Do item.")

    delete_choice = input("Do you want to delete a To-Do item once it's done? (yes/no): ").lower()

    if delete_choice == "yes":
        todo_id_to_delete = int(input("Enter the ID of the To-Do item you want to delete: "))
        delete_by_id(todo_id_to_delete)
    elif delete_choice == "no":
        print("No To-Do item will be deleted.")
    else:
        print("Invalid input. No To-Do item will be deleted.")


initTodos()
create_todo()


Created
 {'id': 9, 'title': 'j', 'description': 'a', 'completed': False}
To-Do item with ID 5 has been deleted.


In [21]:
import sqlite3

def schema():
    connection = sqlite3.connect('database1.db')
    cursor = connection.cursor()

    cursor.execute('SELECT * FROM todos')

    todos = []
    for row in cursor:
        todo = {
            "id": row[0],
            "title": row[1],
            "description": row[2],
            "completed": bool(row[3]),
        }
        todos.append(todo)

    cursor.close()
    connection.close()

    return todos

print(schema())


[{'id': 1, 'title': 'hav fun', 'description': 'ok', 'completed': False}]


## CURSOR TO A DIFF DATABASE

In [7]:
import sqlite3

def create_connection():
    conn = sqlite3.connect("database2.db")
    return conn

def create_todo_table(conn):
    cursor = conn.cursor()
    cursor.execute('''CREATE TABLE IF NOT EXISTS todos (
                        id INTEGER PRIMARY KEY,
                        title TEXT NOT NULL,
                        description TEXT,
                        completed INTEGER NOT NULL DEFAULT 0)''')
    conn.commit()

def create_todo_with_cursor(conn, title, description=None, completed=0):
    cursor = conn.cursor()
    cursor.execute("INSERT INTO todos (title, description, completed) VALUES (?, ?, ?)", (title, description, completed))
    conn.commit()
    return cursor.lastrowid

def read_todo_with_cursor(conn, todo_id):
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM todos WHERE id=?", (todo_id,))
    row = cursor.fetchone()
    return row

def update_todo_with_cursor(conn, todo_id, title=None, description=None, completed=None):
    cursor = conn.cursor()

    if title is not None:
        cursor.execute("UPDATE todos SET title=? WHERE id=?", (title, todo_id))

    if description is not None:
        cursor.execute("UPDATE todos SET description=? WHERE id=?", (description, todo_id))

    if completed is not None:
        cursor.execute("UPDATE todos SET completed=? WHERE id=?", (int(completed), todo_id))

    conn.commit()

def delete_todo_with_cursor(conn, todo_id):
    cursor = conn.cursor()
    cursor.execute("DELETE FROM todos WHERE id=?", (todo_id,))
    conn.commit()

def main():
    connection = create_connection()
    create_todo_table(connection)

    while True:
        print("Choose an operation:")
        print("1. Create a new To-Do item")
        print("2. Read a To-Do item by ID")
        print("3. Update a To-Do item")
        print("4. Delete a To-Do item")
        print("5. Exit")

        choice = int(input("Enter the number of your choice: "))

        if choice == 1:
            title = input("Enter the title of the To-Do item: ")
            description = input("Enter a description for the To-Do item (optional): ")
            if description == "":
                description = None
            todo_id = create_todo_with_cursor(connection, title, description)
            print(f"To-Do item created with ID {todo_id}")

        elif choice == 2:
            todo_id = int(input("Enter the ID of the To-Do item you want to read: "))
            todo = read_todo_with_cursor(connection, todo_id)
            print(todo)

        elif choice == 3:
            todo_id = int(input("Enter the ID of the To-Do item you want to update: "))
            title = input("Enter the new title for the To-Do item (leave empty to keep the current title): ")
            description = input("Enter the new description for the To-Do item (leave empty to keep the current description): ")
            completed_str = input("Enter the new completed status for the To-Do item (yes/no, leave empty to keep the current status): ")

            if title == "":
                title = None
            if description == "":
                description = None
            if completed_str == "":
                completed = None
            else:
                completed = completed_str.lower() == "yes"
 
            update_todo_with_cursor(connection, todo_id, title, description, completed)
            print("To-Do item updated")

        elif choice == 4:
            todo_id = int(input("Enter the ID of the To-Do item you want to delete: "))
            delete_todo_with_cursor(connection, todo_id)
            print("To-Do item deleted")

        elif choice == 5:
            print("Goodbye!")
            break

        else:
            print("Invalid choice, please try again.")


if __name__ == "__main__":
    main()



Choose an operation:
1. Create a new To-Do item
2. Read a To-Do item by ID
3. Update a To-Do item
4. Delete a To-Do item
5. Exit
To-Do item created with ID 3
Choose an operation:
1. Create a new To-Do item
2. Read a To-Do item by ID
3. Update a To-Do item
4. Delete a To-Do item
5. Exit


ValueError: invalid literal for int() with base 10: ''